In [ ]:
# =============================================================================
# 04_MODELING_EXPERIMENTS.IPYNB
# L&T Finance Pearl Challenge - Farmer Income Prediction
# Objective: Achieve MAPE < 18% with focus on income range balance
# =============================================================================

# CHUNK 1: INITIAL SETUP & IMPORTS

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
import json
import pickle
from pathlib import Path
from datetime import datetime
import time

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

# Set random seeds for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("🚀 L&T Finance Pearl Challenge - Modeling Experiments")
print("=" * 60)
print(f"📅 Execution started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🎯 Objective: Achieve MAPE < 18% with income range optimization")
print(f"🎲 Random State: {RANDOM_STATE}")
print("=" * 60)

# Core ML Libraries
from sklearn.model_selection import (
    cross_val_score, StratifiedKFold, KFold, 
    GridSearchCV, RandomizedSearchCV
)
from sklearn.metrics import (
    mean_absolute_percentage_error, mean_absolute_error, 
    mean_squared_error, r2_score
)
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler

print("✅ Core ML libraries imported successfully")

# Advanced ML Libraries  
try:
    import lightgbm as lgb
    print("✅ LightGBM imported successfully")
    LIGHTGBM_AVAILABLE = True
except ImportError:
    print("❌ LightGBM not available")
    LIGHTGBM_AVAILABLE = False

try:
    import xgboost as xgb
    print("✅ XGBoost imported successfully") 
    XGBOOST_AVAILABLE = True
except ImportError:
    print("❌ XGBoost not available")
    XGBOOST_AVAILABLE = False

try:
    import catboost as cb
    print("✅ CatBoost imported successfully")
    CATBOOST_AVAILABLE = True
except ImportError:
    print("❌ CatBoost not available")
    CATBOOST_AVAILABLE = False

# Hyperparameter Optimization
try:
    import optuna
    print("✅ Optuna imported successfully")
    OPTUNA_AVAILABLE = True
    # Suppress optuna logs for cleaner output
    optuna.logging.set_verbosity(optuna.logging.WARNING)
except ImportError:
    print("❌ Optuna not available - using GridSearchCV fallback")
    OPTUNA_AVAILABLE = False

print("\n📋 Library Availability Summary:")
print(f"   🔹 LightGBM: {'✅' if LIGHTGBM_AVAILABLE else '❌'}")
print(f"   🔹 XGBoost: {'✅' if XGBOOST_AVAILABLE else '❌'}")
print(f"   🔹 CatBoost: {'✅' if CATBOOST_AVAILABLE else '❌'}")
print(f"   🔹 Optuna: {'✅' if OPTUNA_AVAILABLE else '❌'}")
print("=" * 60)

In [ ]:
# =============================================================================
# CHUNK 2: DIRECTORY SETUP & DATA LOADING
# =============================================================================

# Set up project directories
BASE_DIR = Path("../")
DATA_DIR = BASE_DIR / "data"
ENGINEERED_DIR = DATA_DIR / "feature_engineered"
RESULTS_DIR = BASE_DIR / "results"
MODELS_DIR = BASE_DIR / "models"

# Create directories if they don't exist
MODELS_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print("📁 Directory Structure:")
print(f"   🔹 Base Directory: {BASE_DIR}")
print(f"   🔹 Data Directory: {ENGINEERED_DIR}")
print(f"   🔹 Models Directory: {MODELS_DIR}")
print(f"   🔹 Results Directory: {RESULTS_DIR}")

# Verify required files exist
required_files = [
    "X_train_eng.npy", "X_val_eng.npy", "X_test_eng.npy",
    "y_train_eng.npy", "y_val_eng.npy", "feature_metadata.json"
]

print("\n🔍 Checking required files:")
missing_files = []
for file in required_files:
    file_path = ENGINEERED_DIR / file
    if file_path.exists():
        print(f"   ✅ {file}")
    else:
        print(f"   ❌ {file} - MISSING!")
        missing_files.append(file)

if missing_files:
    print(f"\n❌ ERROR: Missing files: {missing_files}")
    print("Please run preprocessing notebooks first!")
    raise FileNotFoundError("Required preprocessed files not found")

print("\n📊 Loading preprocessed datasets...")

# Load training and validation data
print("   🔄 Loading X_train...")
X_train = np.load(ENGINEERED_DIR / "X_train_eng.npy").astype(np.float32)
print("   🔄 Loading X_val...")
X_val = np.load(ENGINEERED_DIR / "X_val_eng.npy").astype(np.float32)
print("   🔄 Loading X_test...")
X_test = np.load(ENGINEERED_DIR / "X_test_eng.npy").astype(np.float32)
print("   🔄 Loading y_train...")
y_train = np.load(ENGINEERED_DIR / "y_train_eng.npy").astype(np.float32)
print("   🔄 Loading y_val...")
y_val = np.load(ENGINEERED_DIR / "y_val_eng.npy").astype(np.float32)

# Load feature metadata
print("   🔄 Loading feature metadata...")
with open(ENGINEERED_DIR / "feature_metadata.json", 'r') as f:
    feature_metadata = json.load(f)

feature_names = feature_metadata['final_feature_names']

# Memory usage check
def get_memory_usage():
    """Calculate memory usage of loaded arrays"""
    arrays = [X_train, X_val, X_test, y_train, y_val]
    total_memory = sum(arr.nbytes for arr in arrays) / (1024**2)  # MB
    return total_memory

memory_mb = get_memory_usage()

print(f"\n✅ Data loading completed successfully!")
print("=" * 60)
print("📊 DATASET SUMMARY:")
print(f"   🔹 X_train shape: {X_train.shape}")
print(f"   🔹 X_val shape: {X_val.shape}")
print(f"   🔹 X_test shape: {X_test.shape}")
print(f"   🔹 y_train shape: {y_train.shape}")
print(f"   🔹 y_val shape: {y_val.shape}")
print(f"   🔹 Feature count: {len(feature_names)}")
print(f"   🔹 Total memory usage: {memory_mb:.1f} MB")
print("=" * 60)

In [ ]:
# =============================================================================
# CHUNK 3: MANDATORY LOG TRANSFORMATION & INCOME RANGE SETUP
# =============================================================================

print("🎯 CRITICAL: Applying mandatory log1p transformation")
print("   📈 Problem: 2-5L income range shows 41.43% MAPE (vs target <18%)")
print("   💡 Solution: Log transformation + range-specific modeling")

# MANDATORY: Apply log1p transformation to target variables
print("\n🔄 Applying log1p transformation...")
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

print(f"   ✅ y_train_log: {y_train_log.shape} | Range: [{y_train_log.min():.3f}, {y_train_log.max():.3f}]")
print(f"   ✅ y_val_log: {y_val_log.shape} | Range: [{y_val_log.min():.3f}, {y_val_log.max():.3f}]")

# Create income range masks for specialized analysis
print("\n🎯 Creating income range masks...")
income_ranges = {
    'low': (y_train >= 200000) & (y_train < 500000),    # 2-5 Lakhs
    'mid': (y_train >= 500000) & (y_train < 1000000),   # 5-10 Lakhs  
    'high': y_train >= 1000000                          # 10+ Lakhs
}

income_ranges_val = {
    'low': (y_val >= 200000) & (y_val < 500000),
    'mid': (y_val >= 500000) & (y_val < 1000000),
    'high': y_val >= 1000000
}

# Analyze income distribution
print("\n📊 INCOME RANGE ANALYSIS:")
print("Training Set:")
for range_name, mask in income_ranges.items():
    count = mask.sum()
    percentage = (count / len(y_train)) * 100
    avg_income = y_train[mask].mean() if count > 0 else 0
    print(f"   🔹 {range_name.upper():>4} (2-5L/5-10L/10L+): {count:>6,} samples ({percentage:>5.1f}%) | Avg: ₹{avg_income:>8,.0f}")

print("\nValidation Set:")
for range_name, mask in income_ranges_val.items():
    count = mask.sum()
    percentage = (count / len(y_val)) * 100
    avg_income = y_val[mask].mean() if count > 0 else 0
    print(f"   🔹 {range_name.upper():>4} (2-5L/5-10L/10L+): {count:>6,} samples ({percentage:>5.1f}%) | Avg: ₹{avg_income:>8,.0f}")

# Create sample weights (3x weight for 2-5L range)
print("\n⚖️  Creating sample weights for range balancing...")
sample_weights = np.ones(len(y_train_log))
sample_weights[income_ranges['low']] = 3.0  # 3x weight for 2-5L range

print(f"   ✅ Sample weights created: {len(sample_weights)} weights")
print(f"   🔹 Standard weight (5-10L, 10L+): 1.0 ({(sample_weights == 1.0).sum():,} samples)")
print(f"   🔹 Enhanced weight (2-5L): 3.0 ({(sample_weights == 3.0).sum():,} samples)")
print(f"   🔹 Total weighted samples equivalent: {sample_weights.sum():,.0f}")

# Target variable statistics in both scales
print("\n📈 TARGET VARIABLE STATISTICS:")
print("Original Scale (₹):")
print(f"   🔹 Training   | Mean: ₹{y_train.mean():>8,.0f} | Std: ₹{y_train.std():>8,.0f} | Range: [₹{y_train.min():>7,.0f}, ₹{y_train.max():>9,.0f}]")
print(f"   🔹 Validation | Mean: ₹{y_val.mean():>8,.0f} | Std: ₹{y_val.std():>8,.0f} | Range: [₹{y_val.min():>7,.0f}, ₹{y_val.max():>9,.0f}]")

print("\nLog-Transformed Scale:")
print(f"   🔹 Training   | Mean: {y_train_log.mean():>6.3f} | Std: {y_train_log.std():>6.3f} | Range: [{y_train_log.min():>6.3f}, {y_train_log.max():>6.3f}]")
print(f"   🔹 Validation | Mean: {y_val_log.mean():>6.3f} | Std: {y_val_log.std():>6.3f} | Range: [{y_val_log.min():>6.3f}, {y_val_log.max():>6.3f}]")

print("=" * 60)

In [ ]:
# =============================================================================
# CHUNK 4: ENHANCED EVALUATION FRAMEWORK
# =============================================================================

print("📊 Setting up enhanced evaluation framework with income range analysis")

def calculate_mape(y_true, y_pred):
    """Calculate Mean Absolute Percentage Error with safety checks"""
    # Avoid division by zero
    mask = y_true != 0
    if not mask.any():
        return np.inf
    
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    return mape

def evaluate_with_range_analysis(y_true_log, y_pred_log, income_ranges_mask=None, 
                                dataset_name="Dataset", return_details=False):
    """
    Comprehensive evaluation with income range breakdown
    
    Parameters:
    - y_true_log: True values in log space
    - y_pred_log: Predicted values in log space  
    - income_ranges_mask: Dict with range masks for original scale
    - dataset_name: Name for reporting
    - return_details: Whether to return detailed metrics
    """
    
    # Convert from log space to original scale
    y_true_orig = np.expm1(y_true_log)
    y_pred_orig = np.expm1(y_pred_log)
    
    # Overall metrics
    overall_mape = calculate_mape(y_true_orig, y_pred_orig)
    overall_mae = mean_absolute_error(y_true_orig, y_pred_orig)
    overall_rmse = np.sqrt(mean_squared_error(y_true_orig, y_pred_orig))
    overall_r2 = r2_score(y_true_orig, y_pred_orig)
    
    print(f"\n📊 {dataset_name} PERFORMANCE ANALYSIS:")
    print(f"   🎯 Overall MAPE: {overall_mape:.2f}%")
    print(f"   📏 Overall MAE:  ₹{overall_mae:,.0f}")
    print(f"   📐 Overall RMSE: ₹{overall_rmse:,.0f}")
    print(f"   📈 Overall R²:   {overall_r2:.4f}")
    
    # Range-specific analysis if masks provided
    range_metrics = {}
    if income_ranges_mask is not None:
        print(f"\n🎯 INCOME RANGE BREAKDOWN:")
        
        range_names = {'low': '2-5L', 'mid': '5-10L', 'high': '10L+'}
        
        for range_key, mask in income_ranges_mask.items():
            if mask.sum() > 0:
                range_mape = calculate_mape(y_true_orig[mask], y_pred_orig[mask])
                range_mae = mean_absolute_error(y_true_orig[mask], y_pred_orig[mask])
                range_count = mask.sum()
                
                range_metrics[range_key] = {
                    'mape': range_mape,
                    'mae': range_mae,
                    'count': range_count
                }
                
                status = "✅" if range_mape < 20 else "⚠️" if range_mape < 25 else "❌"
                print(f"   {status} {range_names[range_key]:>4}: {range_mape:>6.2f}% MAPE | ₹{range_mae:>8,.0f} MAE | {range_count:>6,} samples")
            else:
                print(f"   ⭕ {range_names[range_key]:>4}: No samples in range")
    
    if return_details:
        return {
            'overall_mape': overall_mape,
            'overall_mae': overall_mae, 
            'overall_rmse': overall_rmse,
            'overall_r2': overall_r2,
            'range_metrics': range_metrics
        }
    else:
        return overall_mape

# Cross-validation setup with stratification
def create_stratified_cv(y_train, n_splits=5):
    """Create stratified CV splits based on income ranges"""
    
    # Create income bins for stratification
    income_bins = np.digitize(y_train, bins=[200000, 500000, 1000000, 2000000])
    
    # Use StratifiedKFold with income bins
    cv_splitter = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    
    print(f"✅ Created {n_splits}-fold stratified CV based on income ranges")
    print(f"   🎯 Bin distribution: {np.bincount(income_bins)}")
    
    return cv_splitter, income_bins

# Model performance tracking
class ModelTracker:
    """Track model performance across experiments"""
    
    def __init__(self):
        self.results = []
        
    def add_result(self, model_name, overall_mape, range_metrics=None, 
                  training_time=0, memory_usage=0, hyperparams=None):
        """Add model result to tracking"""
        
        result = {
            'model_name': model_name,
            'overall_mape': overall_mape,
            'training_time': training_time,
            'memory_usage': memory_usage,
            'hyperparams': hyperparams or {},
            'timestamp': datetime.now()
        }
        
        if range_metrics:
            result.update({
                'low_mape': range_metrics.get('low', {}).get('mape', np.nan),
                'mid_mape': range_metrics.get('mid', {}).get('mape', np.nan), 
                'high_mape': range_metrics.get('high', {}).get('mape', np.nan)
            })
            
        self.results.append(result)
        
    def get_summary(self):
        """Get performance summary table"""
        if not self.results:
            print("No results to display")
            return None
            
        df = pd.DataFrame(self.results)
        return df.sort_values('overall_mape')
    
    def get_best_model(self):
        """Get best performing model"""
        if not self.results:
            return None
        return min(self.results, key=lambda x: x['overall_mape'])

# Initialize model tracker
model_tracker = ModelTracker()

# Visualization functions
def plot_predictions_vs_actual(y_true, y_pred, title="Predictions vs Actual", 
                              income_ranges_mask=None):
    """Plot predictions vs actual values with range highlighting"""
    
    plt.figure(figsize=(12, 8))
    
    # Convert to millions for better readability
    y_true_millions = y_true / 1000000
    y_pred_millions = y_pred / 1000000
    
    if income_ranges_mask is not None:
        # Plot different ranges in different colors
        colors = {'low': 'red', 'mid': 'blue', 'high': 'green'}
        labels = {'low': '2-5L', 'mid': '5-10L', 'high': '10L+'}
        
        for range_key, mask in income_ranges_mask.items():
            if mask.sum() > 0:
                plt.scatter(y_true_millions[mask], y_pred_millions[mask], 
                          alpha=0.6, c=colors[range_key], label=labels[range_key], s=20)
    else:
        plt.scatter(y_true_millions, y_pred_millions, alpha=0.6, s=20)
    
    # Perfect prediction line
    max_val = max(y_true_millions.max(), y_pred_millions.max())
    min_val = min(y_true_millions.min(), y_pred_millions.min())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.8, linewidth=2)
    
    plt.xlabel('Actual Income (₹ Millions)')
    plt.ylabel('Predicted Income (₹ Millions)')
    plt.title(title)
    
    if income_ranges_mask is not None:
        plt.legend()
    
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Initialize CV splitter
cv_splitter, income_bins = create_stratified_cv(y_train, n_splits=5)

print("✅ Enhanced evaluation framework setup complete!")
print("=" * 60)

In [ ]:
# =============================================================================
# CHUNK 5: BASELINE MODELS
# =============================================================================

print("📈 Training baseline models to establish performance benchmarks")
print("🎯 All models will use log-transformed targets with sample weighting")

# Simple baseline predictions
print("\n🔵 SIMPLE BASELINES:")

# Mean baseline
mean_baseline_log = np.full_like(y_val_log, y_train_log.mean())
mean_mape = evaluate_with_range_analysis(y_val_log, mean_baseline_log, 
                                        income_ranges_val, "Mean Baseline")
model_tracker.add_result("Mean Baseline", mean_mape)

# Median baseline  
median_baseline_log = np.full_like(y_val_log, np.median(y_train_log))
median_mape = evaluate_with_range_analysis(y_val_log, median_baseline_log,
                                          income_ranges_val, "Median Baseline")
model_tracker.add_result("Median Baseline", median_mape)

print("\n" + "="*60)

# Linear models with regularization
print("🔵 LINEAR MODEL BASELINES:")

# Ridge Regression
print("\n📊 Training Ridge Regression...")
start_time = time.time()

ridge_model = Ridge(alpha=1.0, random_state=RANDOM_STATE)
ridge_model.fit(X_train, y_train_log, sample_weight=sample_weights)

ridge_pred_log = ridge_model.predict(X_val)
ridge_time = time.time() - start_time

ridge_metrics = evaluate_with_range_analysis(y_val_log, ridge_pred_log, 
                                           income_ranges_val, "Ridge Regression", 
                                           return_details=True)
model_tracker.add_result("Ridge Regression", ridge_metrics['overall_mape'], 
                        ridge_metrics['range_metrics'], ridge_time)

# Lasso Regression
print("\n📊 Training Lasso Regression...")
start_time = time.time()

lasso_model = Lasso(alpha=0.1, random_state=RANDOM_STATE, max_iter=2000)
lasso_model.fit(X_train, y_train_log, sample_weight=sample_weights)

lasso_pred_log = lasso_model.predict(X_val)
lasso_time = time.time() - start_time

lasso_metrics = evaluate_with_range_analysis(y_val_log, lasso_pred_log,
                                           income_ranges_val, "Lasso Regression",
                                           return_details=True)
model_tracker.add_result("Lasso Regression", lasso_metrics['overall_mape'],
                        lasso_metrics['range_metrics'], lasso_time)

print("\n" + "="*60)

# Tree-based baselines
print("🔵 TREE-BASED BASELINES:")

# Random Forest baseline
print("\n📊 Training Random Forest Baseline...")
start_time = time.time()

rf_baseline = RandomForestRegressor(
    n_estimators=100,
    max_depth=10, 
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

rf_baseline.fit(X_train, y_train_log, sample_weight=sample_weights)
rf_pred_log = rf_baseline.predict(X_val)
rf_time = time.time() - start_time

rf_metrics = evaluate_with_range_analysis(y_val_log, rf_pred_log,
                                        income_ranges_val, "Random Forest Baseline",
                                        return_details=True)
model_tracker.add_result("Random Forest Baseline", rf_metrics['overall_mape'],
                        rf_metrics['range_metrics'], rf_time)

# Extra Trees baseline
print("\n📊 Training Extra Trees Baseline...")
start_time = time.time()

et_baseline = ExtraTreesRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5, 
    min_samples_leaf=2,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

et_baseline.fit(X_train, y_train_log, sample_weight=sample_weights)
et_pred_log = et_baseline.predict(X_val)
et_time = time.time() - start_time

et_metrics = evaluate_with_range_analysis(y_val_log, et_pred_log,
                                        income_ranges_val, "Extra Trees Baseline", 
                                        return_details=True)
model_tracker.add_result("Extra Trees Baseline", et_metrics['overall_mape'],
                        et_metrics['range_metrics'], et_time)

print("\n" + "="*60)

# Gradient boosting baselines (if available)
if XGBOOST_AVAILABLE:
    print("🔵 GRADIENT BOOSTING BASELINE:")
    print("\n📊 Training XGBoost Baseline...")
    start_time = time.time()
    
    xgb_baseline = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        verbosity=0
    )
    
    xgb_baseline.fit(X_train, y_train_log, sample_weight=sample_weights)
    xgb_pred_log = xgb_baseline.predict(X_val)
    xgb_time = time.time() - start_time
    
    xgb_metrics = evaluate_with_range_analysis(y_val_log, xgb_pred_log,
                                             income_ranges_val, "XGBoost Baseline",
                                             return_details=True)
    model_tracker.add_result("XGBoost Baseline", xgb_metrics['overall_mape'],
                            xgb_metrics['range_metrics'], xgb_time)

print("\n" + "="*60)

# Baseline results summary
print("📊 BASELINE MODELS SUMMARY:")
baseline_summary = model_tracker.get_summary()
if baseline_summary is not None:
    print("\nTop 5 Baseline Models:")
    display_cols = ['model_name', 'overall_mape', 'low_mape', 'mid_mape', 'high_mape', 'training_time']
    available_cols = [col for col in display_cols if col in baseline_summary.columns]
    print(baseline_summary[available_cols].head().to_string(index=False, float_format='%.2f'))

# Best baseline identification
best_baseline = model_tracker.get_best_model()
if best_baseline:
    print(f"\n🏆 BEST BASELINE MODEL: {best_baseline['model_name']}")
    print(f"   🎯 Overall MAPE: {best_baseline['overall_mape']:.2f}%")
    if 'low_mape' in best_baseline:
        print(f"   🔴 2-5L MAPE: {best_baseline['low_mape']:.2f}%") 
        print(f"   🔵 5-10L MAPE: {best_baseline['mid_mape']:.2f}%")
        print(f"   🟢 10L+ MAPE: {best_baseline['high_mape']:.2f}%")

print("\n✅ Baseline models training complete!")
print("🎯 Target for advanced models: Beat best baseline performance")
print("🔥 Critical focus: Reduce 2-5L range MAPE significantly")
print("=" * 60)

In [ ]:
# =============================================================================
# CHUNK 6: ADVANCED MODEL 1 - LIGHTGBM WITH OPTUNA
# =============================================================================

if LIGHTGBM_AVAILABLE and OPTUNA_AVAILABLE:
    print("🚀 Training LightGBM with Optuna hyperparameter optimization")
    print("🎯 Focus: Minimize overall MAPE with 2-5L range penalty")
    
    def objective_lightgbm(trial):
        """Optuna objective function for LightGBM optimization"""
        
        # Define hyperparameter search space
        params = {
            'objective': 'regression',
            'metric': 'mae',
            'boosting_type': 'gbdt',
            'num_leaves': trial.suggest_int('num_leaves', 20, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
            'verbosity': -1,
            'random_state': RANDOM_STATE,
            'n_jobs': -1
        }
        
        # Add GPU support if available
        try:
            params['device'] = 'gpu'
            params['gpu_platform_id'] = 0
            params['gpu_device_id'] = 0
        except:
            params['device'] = 'cpu'
        
        # Cross-validation with income range evaluation
        cv_scores = []
        cv_range_penalties = []
        
        for train_idx, val_idx in cv_splitter.split(X_train, income_bins):
            X_cv_train, X_cv_val = X_train[train_idx], X_train[val_idx]
            y_cv_train, y_cv_val = y_train_log[train_idx], y_train_log[val_idx]
            weights_cv_train = sample_weights[train_idx]
            
            # Create LightGBM datasets
            train_data = lgb.Dataset(X_cv_train, label=y_cv_train, weight=weights_cv_train)
            
            # Train model
            model = lgb.train(
                params,
                train_data,
                num_boost_round=1000,
                valid_sets=[train_data],
                callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)]
            )
            
            # Predict and evaluate
            y_pred_log = model.predict(X_cv_val, num_iteration=model.best_iteration)
            
            # Convert to original scale for MAPE calculation
            y_true_orig = np.expm1(y_cv_val)
            y_pred_orig = np.expm1(y_pred_log)
            
            # Calculate overall MAPE
            overall_mape = calculate_mape(y_true_orig, y_pred_orig)
            cv_scores.append(overall_mape)
            
            # Calculate 2-5L range penalty (if samples exist)
            low_mask = (y_true_orig >= 200000) & (y_true_orig < 500000)
            if low_mask.sum() > 0:
                low_mape = calculate_mape(y_true_orig[low_mask], y_pred_orig[low_mask])
                # Penalty: heavily weight 2-5L performance
                range_penalty = max(0, low_mape - 25) * 2  # Penalty if >25% MAPE
                cv_range_penalties.append(range_penalty)
            else:
                cv_range_penalties.append(0)
        
        # Final objective: overall MAPE + range penalty
        mean_mape = np.mean(cv_scores)
        mean_penalty = np.mean(cv_range_penalties)
        objective_value = mean_mape + mean_penalty
        
        return objective_value
    
    # Run Optuna optimization
    print("\n🔄 Starting Optuna optimization (Target: 200 trials)...")
    print("   💡 Objective: Minimize (Overall MAPE + 2-5L Range Penalty)")
    
    study_lgb = optuna.create_study(direction='minimize', study_name='lightgbm_optimization')
    
    start_time = time.time()
    study_lgb.optimize(objective_lightgbm, n_trials=200, timeout=1800)  # 30 min timeout
    optimization_time = time.time() - start_time
    
    print(f"\n✅ Optuna optimization completed in {optimization_time/60:.1f} minutes")
    print(f"🏆 Best objective value: {study_lgb.best_value:.3f}")
    print(f"🎯 Best trial number: {study_lgb.best_trial.number}")
    
    # Train final model with best parameters
    print("\n🔄 Training final LightGBM model with best parameters...")
    
    best_params = study_lgb.best_params.copy()
    best_params.update({
        'objective': 'regression',
        'metric': 'mae',
        'boosting_type': 'gbdt',
        'verbosity': -1,
        'random_state': RANDOM_STATE,
        'n_jobs': -1
    })
    
    # Add GPU support
    try:
        best_params['device'] = 'gpu'
        best_params['gpu_platform_id'] = 0
        best_params['gpu_device_id'] = 0
        print("   ⚡ Using GPU acceleration")
    except:
        best_params['device'] = 'cpu'
        print("   🖥️  Using CPU training")
    
    # Create datasets
    train_data = lgb.Dataset(X_train, label=y_train_log, weight=sample_weights)
    val_data = lgb.Dataset(X_val, label=y_val_log, reference=train_data)
    
    # Train final model
    start_time = time.time()
    lgb_final = lgb.train(
        best_params,
        train_data,
        num_boost_round=2000,
        valid_sets=[train_data, val_data],
        valid_names=['train', 'val'],
        callbacks=[lgb.early_stopping(150), lgb.log_evaluation(100)]
    )
    training_time = time.time() - start_time
    
    # Generate predictions
    lgb_pred_log = lgb_final.predict(X_val, num_iteration=lgb_final.best_iteration)
    
    # Comprehensive evaluation
    print(f"\n📊 Final LightGBM model training completed in {training_time:.1f} seconds")
    lgb_metrics = evaluate_with_range_analysis(y_val_log, lgb_pred_log,
                                             income_ranges_val, "LightGBM Optimized",
                                             return_details=True)
    
    # Track results
    model_tracker.add_result("LightGBM Optimized", lgb_metrics['overall_mape'],
                            lgb_metrics['range_metrics'], training_time,
                            hyperparams=best_params)
    
    # Feature importance analysis
    print("\n📊 LightGBM Feature Importance (Top 15):")
    feature_importance = lgb_final.feature_importance(importance_type='gain')
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)
    
    print(importance_df.head(15).to_string(index=False))
    
    # Save model
    model_path = MODELS_DIR / "lightgbm_optimized.pkl"
    with open(model_path, 'wb') as f:
        pickle.dump({
            'model': lgb_final,
            'params': best_params,
            'feature_names': feature_names,
            'cv_score': study_lgb.best_value,
            'feature_importance': importance_df
        }, f)
    
    print(f"\n💾 LightGBM model saved to: {model_path}")
    
    # Visualization of hyperparameter optimization
    print("\n📈 Optimization Analysis:")
    print(f"   🔹 Total trials: {len(study_lgb.trials)}")
    print(f"   🔹 Best trial: {study_lgb.best_trial.number}")
    print(f"   🔹 Best parameters:")
    for param, value in study_lgb.best_params.items():
        print(f"      • {param}: {value}")
    
    print("\n✅ LightGBM optimization and training complete!")
    
else:
    print("❌ LightGBM or Optuna not available - skipping LightGBM optimization")
    lgb_metrics = None

print("=" * 60)

In [ ]:
# =============================================================================
# CHUNK 7: ADVANCED MODEL 2 - XGBOOST WITH OPTUNA
# =============================================================================

if XGBOOST_AVAILABLE and OPTUNA_AVAILABLE:
    print("🚀 Training XGBoost with Optuna hyperparameter optimization")
    print("🎯 Focus: Minimize overall MAPE with enhanced 2-5L range penalty")
    
    def objective_xgboost(trial):
        """Optuna objective function for XGBoost optimization"""
        
        # Define hyperparameter search space
        params = {
            'objective': 'reg:squarederror',
            'eval_metric': 'mae',
            'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
            'gamma': trial.suggest_float('gamma', 0.0, 5.0),
            'random_state': RANDOM_STATE,
            'n_jobs': -1,
            'verbosity': 0
        }
        
        # Add GPU support if available
        try:
            params['tree_method'] = 'gpu_hist'
            params['gpu_id'] = 0
        except:
            params['tree_method'] = 'hist'
        
        # Cross-validation with income range evaluation
        cv_scores = []
        cv_range_penalties = []
        
        for train_idx, val_idx in cv_splitter.split(X_train, income_bins):
            X_cv_train, X_cv_val = X_train[train_idx], X_train[val_idx]
            y_cv_train, y_cv_val = y_train_log[train_idx], y_train_log[val_idx]
            weights_cv_train = sample_weights[train_idx]
            
            # Train model
            model = xgb.XGBRegressor(**params)
            model.fit(
                X_cv_train, y_cv_train,
                sample_weight=weights_cv_train,
                eval_set=[(X_cv_train, y_cv_train)],
                early_stopping_rounds=100,
                verbose=False
            )
            
            # Predict and evaluate
            y_pred_log = model.predict(X_cv_val)
            
            # Convert to original scale for MAPE calculation
            y_true_orig = np.expm1(y_cv_val)
            y_pred_orig = np.expm1(y_pred_log)
            
            # Calculate overall MAPE
            overall_mape = calculate_mape(y_true_orig, y_pred_orig)
            cv_scores.append(overall_mape)
            
            # Calculate 2-5L range penalty (enhanced)
            low_mask = (y_true_orig >= 200000) & (y_true_orig < 500000)
            if low_mask.sum() > 0:
                low_mape = calculate_mape(y_true_orig[low_mask], y_pred_orig[low_mask])
                # Enhanced penalty: more aggressive for 2-5L range
                range_penalty = max(0, low_mape - 20) * 3  # Penalty if >20% MAPE
                cv_range_penalties.append(range_penalty)
            else:
                cv_range_penalties.append(0)
        
        # Final objective: overall MAPE + enhanced range penalty
        mean_mape = np.mean(cv_scores)
        mean_penalty = np.mean(cv_range_penalties)
        objective_value = mean_mape + mean_penalty
        
        return objective_value
    
    # Run Optuna optimization
    print("\n🔄 Starting Optuna optimization (Target: 200 trials)...")
    print("   💡 Objective: Minimize (Overall MAPE + Enhanced 2-5L Range Penalty)")
    
    study_xgb = optuna.create_study(direction='minimize', study_name='xgboost_optimization')
    
    start_time = time.time()
    study_xgb.optimize(objective_xgboost, n_trials=200, timeout=1800)  # 30 min timeout
    optimization_time = time.time() - start_time
    
    print(f"\n✅ Optuna optimization completed in {optimization_time/60:.1f} minutes")
    print(f"🏆 Best objective value: {study_xgb.best_value:.3f}")
    print(f"🎯 Best trial number: {study_xgb.best_trial.number}")
    
    # Train final model with best parameters
    print("\n🔄 Training final XGBoost model with best parameters...")
    
    best_params = study_xgb.best_params.copy()
    best_params.update({
        'objective': 'reg:squarederror',
        'eval_metric': 'mae',
        'random_state': RANDOM_STATE,
        'n_jobs': -1,
        'verbosity': 0
    })
    
    # Add GPU support
    try:
        best_params['tree_method'] = 'gpu_hist'
        best_params['gpu_id'] = 0
        print("   ⚡ Using GPU acceleration")
    except:
        best_params['tree_method'] = 'hist'
        print("   🖥️  Using CPU training")
    
    # Train final model
    start_time = time.time()
    xgb_final = xgb.XGBRegressor(**best_params)
    xgb_final.fit(
        X_train, y_train_log,
        sample_weight=sample_weights,
        eval_set=[(X_train, y_train_log), (X_val, y_val_log)],
        early_stopping_rounds=150,
        verbose=100
    )
    training_time = time.time() - start_time
    
    # Generate predictions
    xgb_pred_log = xgb_final.predict(X_val)
    
    # Comprehensive evaluation
    print(f"\n📊 Final XGBoost model training completed in {training_time:.1f} seconds")
    xgb_metrics = evaluate_with_range_analysis(y_val_log, xgb_pred_log,
                                             income_ranges_val, "XGBoost Optimized",
                                             return_details=True)
    
    # Track results
    model_tracker.add_result("XGBoost Optimized", xgb_metrics['overall_mape'],
                            xgb_metrics['range_metrics'], training_time,
                            hyperparams=best_params)
    
    # Feature importance analysis
    print("\n📊 XGBoost Feature Importance (Top 15):")
    feature_importance = xgb_final.feature_importances_
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)
    
    print(importance_df.head(15).to_string(index=False))
    
    # Training progress visualization
    if hasattr(xgb_final, 'evals_result_'):
        results = xgb_final.evals_result_
        plt.figure(figsize=(12, 4))
        
        plt.subplot(1, 2, 1)
        plt.plot(results['validation_0']['mae'], label='Training MAE')
        plt.plot(results['validation_1']['mae'], label='Validation MAE')
        plt.xlabel('Iteration')
        plt.ylabel('MAE')
        plt.title('XGBoost Training Progress')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.subplot(1, 2, 2)
        plt.bar(range(len(importance_df.head(10))), importance_df.head(10)['importance'])
        plt.xlabel('Feature Rank')
        plt.ylabel('Importance')
        plt.title('Top 10 Feature Importances')
        plt.xticks(range(10), range(1, 11))
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    # Save model
    model_path = MODELS_DIR / "xgboost_optimized.pkl"
    with open(model_path, 'wb') as f:
        pickle.dump({
            'model': xgb_final,
            'params': best_params,
            'feature_names': feature_names,
            'cv_score': study_xgb.best_value,
            'feature_importance': importance_df
        }, f)
    
    print(f"\n💾 XGBoost model saved to: {model_path}")
    
    # Optimization analysis
    print("\n📈 Optimization Analysis:")
    print(f"   🔹 Total trials: {len(study_xgb.trials)}")
    print(f"   🔹 Best trial: {study_xgb.best_trial.number}")
    print(f"   🔹 Best parameters:")
    for param, value in study_xgb.best_params.items():
        print(f"      • {param}: {value}")
    
    # Compare with previous best
    current_best = model_tracker.get_best_model()
    if current_best and current_best['model_name'] == 'XGBoost Optimized':
        print(f"\n🏆 XGBoost is now the best model!")
    
    print("\n✅ XGBoost optimization and training complete!")
    
else:
    print("❌ XGBoost or Optuna not available - skipping XGBoost optimization")
    xgb_metrics = None

print("=" * 60)

In [ ]:
# =============================================================================
# CHUNK 8: ADVANCED MODEL 3 - CATBOOST WITH OPTUNA
# =============================================================================

if CATBOOST_AVAILABLE and OPTUNA_AVAILABLE:
    print("🚀 Training CatBoost with Optuna hyperparameter optimization")
    print("🎯 Focus: Leverage CatBoost's built-in overfitting detection + 2-5L penalty")
    
    def objective_catboost(trial):
        """Optuna objective function for CatBoost optimization"""
        
        # Define hyperparameter search space
        params = {
            'loss_function': 'MAE',
            'iterations': trial.suggest_int('iterations', 500, 3000),
            'depth': trial.suggest_int('depth', 4, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 30),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 50),
            'max_leaves': trial.suggest_int('max_leaves', 16, 512),
            'random_state': RANDOM_STATE,
            'verbose': False,
            'thread_count': -1
        }
        
        # Add GPU support if available
        try:
            params['task_type'] = 'GPU'
            params['gpu_ram_part'] = 0.5
        except:
            params['task_type'] = 'CPU'
        
        # Cross-validation with income range evaluation
        cv_scores = []
        cv_range_penalties = []
        
        for train_idx, val_idx in cv_splitter.split(X_train, income_bins):
            X_cv_train, X_cv_val = X_train[train_idx], X_train[val_idx]
            y_cv_train, y_cv_val = y_train_log[train_idx], y_train_log[val_idx]
            weights_cv_train = sample_weights[train_idx]
            
            # Create CatBoost pools
            train_pool = cb.Pool(X_cv_train, y_cv_train, weight=weights_cv_train)
            val_pool = cb.Pool(X_cv_val, y_cv_val)
            
            # Train model with built-in overfitting detection
            model = cb.CatBoostRegressor(**params)
            model.fit(
                train_pool,
                eval_set=val_pool,
                early_stopping_rounds=100,
                use_best_model=True,
                verbose=False
            )
            
            # Predict and evaluate
            y_pred_log = model.predict(X_cv_val)
            
            # Convert to original scale for MAPE calculation
            y_true_orig = np.expm1(y_cv_val)
            y_pred_orig = np.expm1(y_pred_log)
            
            # Calculate overall MAPE
            overall_mape = calculate_mape(y_true_orig, y_pred_orig)
            cv_scores.append(overall_mape)
            
            # Calculate 2-5L range penalty (most aggressive)
            low_mask = (y_true_orig >= 200000) & (y_true_orig < 500000)
            if low_mask.sum() > 0:
                low_mape = calculate_mape(y_true_orig[low_mask], y_pred_orig[low_mask])
                # Most aggressive penalty for CatBoost
                range_penalty = max(0, low_mape - 18) * 4  # Penalty if >18% MAPE
                cv_range_penalties.append(range_penalty)
            else:
                cv_range_penalties.append(0)
        
        # Final objective: overall MAPE + most aggressive range penalty
        mean_mape = np.mean(cv_scores)
        mean_penalty = np.mean(cv_range_penalties)
        objective_value = mean_mape + mean_penalty
        
        return objective_value
    
    # Run Optuna optimization
    print("\n🔄 Starting Optuna optimization (Target: 150 trials - CatBoost is slower)...")
    print("   💡 Objective: Minimize (Overall MAPE + Most Aggressive 2-5L Penalty)")
    print("   🎯 Using CatBoost's built-in overfitting detection")
    
    study_cb = optuna.create_study(direction='minimize', study_name='catboost_optimization')
    
    start_time = time.time()
    study_cb.optimize(objective_catboost, n_trials=150, timeout=2400)  # 40 min timeout
    optimization_time = time.time() - start_time
    
    print(f"\n✅ Optuna optimization completed in {optimization_time/60:.1f} minutes")
    print(f"🏆 Best objective value: {study_cb.best_value:.3f}")
    print(f"🎯 Best trial number: {study_cb.best_trial.number}")
    
    # Train final model with best parameters
    print("\n🔄 Training final CatBoost model with best parameters...")
    
    best_params = study_cb.best_params.copy()
    best_params.update({
        'loss_function': 'MAE',
        'random_state': RANDOM_STATE,
        'verbose': 100,  # Show progress for final training
        'thread_count': -1
    })
    
    # Add GPU support
    try:
        best_params['task_type'] = 'GPU'
        best_params['gpu_ram_part'] = 0.5
        print("   ⚡ Using GPU acceleration")
    except:
        best_params['task_type'] = 'CPU'
        print("   🖥️  Using CPU training")
    
    # Create pools for training
    train_pool = cb.Pool(X_train, y_train_log, weight=sample_weights)
    val_pool = cb.Pool(X_val, y_val_log)
    
    # Train final model
    start_time = time.time()
    cb_final = cb.CatBoostRegressor(**best_params)
    cb_final.fit(
        train_pool,
        eval_set=val_pool,
        early_stopping_rounds=200,
        use_best_model=True,
        plot=False
    )
    training_time = time.time() - start_time
    
    # Generate predictions
    cb_pred_log = cb_final.predict(X_val)
    
    # Comprehensive evaluation
    print(f"\n📊 Final CatBoost model training completed in {training_time:.1f} seconds")
    cb_metrics = evaluate_with_range_analysis(y_val_log, cb_pred_log,
                                            income_ranges_val, "CatBoost Optimized",
                                            return_details=True)
    
    # Track results
    model_tracker.add_result("CatBoost Optimized", cb_metrics['overall_mape'],
                            cb_metrics['range_metrics'], training_time,
                            hyperparams=best_params)
    
    # Feature importance analysis
    print("\n📊 CatBoost Feature Importance (Top 15):")
    feature_importance = cb_final.get_feature_importance()
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)
    
    print(importance_df.head(15).to_string(index=False))
    
    # Training metrics visualization
    if hasattr(cb_final, 'get_evals_result'):
        evals_result = cb_final.get_evals_result()
        if evals_result:
            plt.figure(figsize=(15, 5))
            
            # Training progress
            plt.subplot(1, 3, 1)
            train_scores = evals_result['learn']['MAE']
            val_scores = evals_result['validation']['MAE']
            iterations = range(len(train_scores))
            
            plt.plot(iterations, train_scores, label='Training MAE', alpha=0.8)
            plt.plot(iterations, val_scores, label='Validation MAE', alpha=0.8)
            plt.xlabel('Iteration')
            plt.ylabel('MAE')
            plt.title('CatBoost Training Progress')
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            # Feature importance
            plt.subplot(1, 3, 2)
            top_features = importance_df.head(10)
            plt.barh(range(len(top_features)), top_features['importance'])
            plt.yticks(range(len(top_features)), 
                      [f[:20] + '...' if len(f) > 20 else f for f in top_features['feature']])
            plt.xlabel('Importance')
            plt.title('Top 10 Feature Importances')
            plt.grid(True, alpha=0.3)
            
            # Learning curve comparison
            plt.subplot(1, 3, 3)
            min_len = min(len(train_scores), len(val_scores))
            diff = np.array(val_scores[:min_len]) - np.array(train_scores[:min_len])
            plt.plot(range(min_len), diff, label='Val - Train MAE', color='red', alpha=0.7)
            plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
            plt.xlabel('Iteration')
            plt.ylabel('MAE Difference')
            plt.title('Overfitting Detection')
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
    
    # Model analysis
    print(f"\n📈 CatBoost Model Analysis:")
    print(f"   🔹 Total iterations: {cb_final.get_best_iteration()}")
    print(f"   🔹 Best iteration: {cb_final.get_best_iteration()}")
    print(f"   🔹 Training score: {cb_final.get_best_score()['learn']['MAE']:.4f}")
    print(f"   🔹 Validation score: {cb_final.get_best_score()['validation']['MAE']:.4f}")
    
    # Save model
    model_path = MODELS_DIR / "catboost_optimized.cbm"
    cb_final.save_model(str(model_path))
    
    # Also save metadata
    metadata_path = MODELS_DIR / "catboost_optimized_metadata.pkl"
    with open(metadata_path, 'wb') as f:
        pickle.dump({
            'params': best_params,
            'feature_names': feature_names,
            'cv_score': study_cb.best_value,
            'feature_importance': importance_df,
            'best_iteration': cb_final.get_best_iteration()
        }, f)
    
    print(f"\n💾 CatBoost model saved to: {model_path}")
    print(f"💾 Metadata saved to: {metadata_path}")
    
    # Optimization analysis
    print("\n📈 Optimization Analysis:")
    print(f"   🔹 Total trials: {len(study_cb.trials)}")
    print(f"   🔹 Best trial: {study_cb.best_trial.number}")
    print(f"   🔹 Best parameters:")
    for param, value in study_cb.best_params.items():
        print(f"      • {param}: {value}")
    
    # Compare with previous best
    current_best = model_tracker.get_best_model()
    if current_best and current_best['model_name'] == 'CatBoost Optimized':
        print(f"\n🏆 CatBoost is now the best model!")
    
    print("\n✅ CatBoost optimization and training complete!")
    
else:
    print("❌ CatBoost or Optuna not available - skipping CatBoost optimization")
    cb_metrics = None

print("=" * 60)

In [ ]:
# =============================================================================
# CHUNK 9: ADVANCED MODEL 4 - EXTRA TREES WITH OPTUNA
# =============================================================================

if OPTUNA_AVAILABLE:
    print("🚀 Training Extra Trees with Optuna hyperparameter optimization")
    print("🎯 Focus: Variance reduction for 2-5L range stability + ensemble diversity")
    
    def objective_extratrees(trial):
        """Optuna objective function for Extra Trees optimization"""
        
        # Define hyperparameter search space
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 5, 30),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', 0.3, 0.5, 0.7, 1.0]),
            'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0.0, 0.01),
            'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
            'random_state': RANDOM_STATE,
            'n_jobs': -1
        }
        
        # Cross-validation with income range evaluation
        cv_scores = []
        cv_range_penalties = []
        cv_range_stds = []  # Track variance for stability
        
        for train_idx, val_idx in cv_splitter.split(X_train, income_bins):
            X_cv_train, X_cv_val = X_train[train_idx], X_train[val_idx]
            y_cv_train, y_cv_val = y_train_log[train_idx], y_train_log[val_idx]
            weights_cv_train = sample_weights[train_idx]
            
            # Train model
            model = ExtraTreesRegressor(**params)
            model.fit(X_cv_train, y_cv_train, sample_weight=weights_cv_train)
            
            # Predict and evaluate
            y_pred_log = model.predict(X_cv_val)
            
            # Convert to original scale for MAPE calculation
            y_true_orig = np.expm1(y_cv_val)
            y_pred_orig = np.expm1(y_pred_log)
            
            # Calculate overall MAPE
            overall_mape = calculate_mape(y_true_orig, y_pred_orig)
            cv_scores.append(overall_mape)
            
            # Calculate 2-5L range penalty and variance
            low_mask = (y_true_orig >= 200000) & (y_true_orig < 500000)
            if low_mask.sum() > 0:
                low_mape = calculate_mape(y_true_orig[low_mask], y_pred_orig[low_mask])
                
                # Standard penalty for 2-5L range
                range_penalty = max(0, low_mape - 22) * 2.5  # Penalty if >22% MAPE
                cv_range_penalties.append(range_penalty)
                
                # Variance penalty - Extra Trees should reduce prediction variance
                low_pred_std = np.std(y_pred_orig[low_mask])
                low_true_std = np.std(y_true_orig[low_mask])
                variance_ratio = low_pred_std / (low_true_std + 1e-8)
                variance_penalty = max(0, variance_ratio - 1.0) * 5  # Penalize high variance
                cv_range_stds.append(variance_penalty)
            else:
                cv_range_penalties.append(0)
                cv_range_stds.append(0)
        
        # Final objective: MAPE + range penalty + variance penalty
        mean_mape = np.mean(cv_scores)
        mean_range_penalty = np.mean(cv_range_penalties)
        mean_variance_penalty = np.mean(cv_range_stds)
        objective_value = mean_mape + mean_range_penalty + mean_variance_penalty
        
        return objective_value
    
    # Run Optuna optimization
    print("\n🔄 Starting Optuna optimization (Target: 100 trials - faster than boosting)...")
    print("   💡 Objective: Minimize (MAPE + 2-5L Range Penalty + Variance Penalty)")
    print("   🎯 Focus on variance reduction and ensemble diversity")
    
    study_et = optuna.create_study(direction='minimize', study_name='extratrees_optimization')
    
    start_time = time.time()
    study_et.optimize(objective_extratrees, n_trials=100, timeout=1200)  # 20 min timeout
    optimization_time = time.time() - start_time
    
    print(f"\n✅ Optuna optimization completed in {optimization_time/60:.1f} minutes")
    print(f"🏆 Best objective value: {study_et.best_value:.3f}")
    print(f"🎯 Best trial number: {study_et.best_trial.number}")
    
    # Train final model with best parameters
    print("\n🔄 Training final Extra Trees model with best parameters...")
    
    best_params = study_et.best_params.copy()
    best_params.update({
        'random_state': RANDOM_STATE,
        'n_jobs': -1
    })
    
    # Train final model
    start_time = time.time()
    et_final = ExtraTreesRegressor(**best_params)
    et_final.fit(X_train, y_train_log, sample_weight=sample_weights)
    training_time = time.time() - start_time
    
    # Generate predictions
    et_pred_log = et_final.predict(X_val)
    
    # Comprehensive evaluation
    print(f"\n📊 Final Extra Trees model training completed in {training_time:.1f} seconds")
    et_metrics = evaluate_with_range_analysis(y_val_log, et_pred_log,
                                            income_ranges_val, "Extra Trees Optimized",
                                            return_details=True)
    
    # Track results
    model_tracker.add_result("Extra Trees Optimized", et_metrics['overall_mape'],
                            et_metrics['range_metrics'], training_time,
                            hyperparams=best_params)
    
    # Feature importance analysis
    print("\n📊 Extra Trees Feature Importance (Top 15):")
    feature_importance = et_final.feature_importances_
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)
    
    print(importance_df.head(15).to_string(index=False))
    
    # Prediction variance analysis
    print("\n📊 Prediction Variance Analysis:")
    y_val_orig = np.expm1(y_val_log)
    et_pred_orig = np.expm1(et_pred_log)
    
    for range_name, mask in income_ranges_val.items():
        if mask.sum() > 0:
            true_std = np.std(y_val_orig[mask])
            pred_std = np.std(et_pred_orig[mask])
            variance_ratio = pred_std / true_std
            range_labels = {'low': '2-5L', 'mid': '5-10L', 'high': '10L+'}
            print(f"   🔹 {range_labels[range_name]:>4}: True Std=₹{true_std:>8,.0f} | Pred Std=₹{pred_std:>8,.0f} | Ratio={variance_ratio:.3f}")
    
    # Ensemble diversity analysis (compare with Random Forest if available)
    rf_available = any(result['model_name'] == 'Random Forest Baseline' for result in model_tracker.results)
    if rf_available:
        print("\n📊 Ensemble Diversity Analysis:")
        # Find Random Forest predictions for comparison
        rf_result = next((r for r in model_tracker.results if r['model_name'] == 'Random Forest Baseline'), None)
        if rf_result:
            print("   🎯 Extra Trees vs Random Forest diversity will be valuable for ensemble")
    
    # Model visualization
    plt.figure(figsize=(15, 5))
    
    # Feature importance
    plt.subplot(1, 3, 1)
    top_features = importance_df.head(10)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), 
              [f[:15] + '...' if len(f) > 15 else f for f in top_features['feature']])
    plt.xlabel('Importance')
    plt.title('Extra Trees: Top 10 Features')
    plt.grid(True, alpha=0.3)
    
    # Predictions vs actual by income range
    plt.subplot(1, 3, 2)
    colors = {'low': 'red', 'mid': 'blue', 'high': 'green'}
    labels = {'low': '2-5L', 'mid': '5-10L', 'high': '10L+'}
    
    for range_name, mask in income_ranges_val.items():
        if mask.sum() > 0:
            y_true_millions = y_val_orig[mask] / 1000000
            y_pred_millions = et_pred_orig[mask] / 1000000
            plt.scatter(y_true_millions, y_pred_millions, 
                       alpha=0.6, c=colors[range_name], label=labels[range_name], s=20)
    
    max_val = max(y_val_orig.max(), et_pred_orig.max()) / 1000000
    min_val = min(y_val_orig.min(), et_pred_orig.min()) / 1000000
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.8, linewidth=2)
    plt.xlabel('Actual Income (₹ Millions)')
    plt.ylabel('Predicted Income (₹ Millions)')
    plt.title('Extra Trees: Predictions vs Actual')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Error distribution by range
    plt.subplot(1, 3, 3)
    errors_by_range = {}
    for range_name, mask in income_ranges_val.items():
        if mask.sum() > 0:
            errors = np.abs(y_val_orig[mask] - et_pred_orig[mask]) / y_val_orig[mask] * 100
            errors_by_range[labels[range_name]] = errors
    
    if errors_by_range:
        plt.boxplot(errors_by_range.values(), labels=errors_by_range.keys())
        plt.ylabel('Absolute Percentage Error (%)')
        plt.title('Error Distribution by Income Range')
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Save model
    model_path = MODELS_DIR / "extratrees_optimized.pkl"
    with open(model_path, 'wb') as f:
        pickle.dump({
            'model': et_final,
            'params': best_params,
            'feature_names': feature_names,
            'cv_score': study_et.best_value,
            'feature_importance': importance_df
        }, f)
    
    print(f"\n💾 Extra Trees model saved to: {model_path}")
    
    # Optimization analysis
    print("\n📈 Optimization Analysis:")
    print(f"   🔹 Total trials: {len(study_et.trials)}")
    print(f"   🔹 Best trial: {study_et.best_trial.number}")
    print(f"   🔹 Best parameters:")
    for param, value in study_et.best_params.items():
        print(f"      • {param}: {value}")
    
    # Compare with previous best
    current_best = model_tracker.get_best_model()
    if current_best and current_best['model_name'] == 'Extra Trees Optimized':
        print(f"\n🏆 Extra Trees is now the best model!")
    
    print("\n✅ Extra Trees optimization and training complete!")
    
else:
    print("❌ Optuna not available - skipping Extra Trees optimization")
    et_metrics = None

print("=" * 60)

In [ ]:
# =============================================================================
# CHUNK 10: MODEL COMPARISON & SELECTION
# =============================================================================

print("📊 Comprehensive Model Performance Comparison & Selection")
print("🎯 Objective: Identify top 3 models for ensemble and final selection")

# Get comprehensive results summary
results_summary = model_tracker.get_summary()

if results_summary is not None and len(results_summary) > 0:
    
    # Display comprehensive comparison table
    print("\n" + "="*80)
    print("📋 COMPLETE MODEL PERFORMANCE COMPARISON")
    print("="*80)
    
    # Select columns for display
    display_columns = ['model_name', 'overall_mape', 'low_mape', 'mid_mape', 'high_mape', 'training_time']
    available_columns = [col for col in display_columns if col in results_summary.columns]
    
    # Format the results for better readability
    display_df = results_summary[available_columns].copy()
    
    # Round numeric columns
    numeric_columns = ['overall_mape', 'low_mape', 'mid_mape', 'high_mape', 'training_time']
    for col in numeric_columns:
        if col in display_df.columns:
            display_df[col] = display_df[col].round(2)
    
    print(display_df.to_string(index=False))
    
    # Statistical analysis of results
    print(f"\n📈 STATISTICAL ANALYSIS:")
    print(f"   🔹 Total models trained: {len(results_summary)}")
    
    if 'overall_mape' in results_summary.columns:
        overall_mapes = results_summary['overall_mape'].dropna()
        print(f"   🔹 Best overall MAPE: {overall_mapes.min():.2f}%")
        print(f"   🔹 Worst overall MAPE: {overall_mapes.max():.2f}%")
        print(f"   🔹 Mean overall MAPE: {overall_mapes.mean():.2f}%")
        print(f"   🔹 MAPE std deviation: {overall_mapes.std():.2f}%")
        
        # Target achievement analysis
        target_achieved = (overall_mapes < 18).sum()
        print(f"   🎯 Models achieving <18% MAPE: {target_achieved}/{len(overall_mapes)}")
    
    # 2-5L range specific analysis
    if 'low_mape' in results_summary.columns:
        low_mapes = results_summary['low_mape'].dropna()
        if len(low_mapes) > 0:
            print(f"\n🔴 2-5L RANGE ANALYSIS:")
            print(f"   🔹 Best 2-5L MAPE: {low_mapes.min():.2f}%")
            print(f"   🔹 Worst 2-5L MAPE: {low_mapes.max():.2f}%")
            print(f"   🔹 Mean 2-5L MAPE: {low_mapes.mean():.2f}%")
            
            # Critical improvement tracking
            baseline_low_mape = 41.43  # From problem context
            best_low_mape = low_mapes.min()
            improvement = baseline_low_mape - best_low_mape
            improvement_pct = (improvement / baseline_low_mape) * 100
            
            print(f"   🚀 Improvement from baseline: -{improvement:.2f}% ({improvement_pct:.1f}% reduction)")
            
            # Target achievement for 2-5L
            low_target_achieved = (low_mapes < 25).sum()
            print(f"   🎯 Models achieving <25% MAPE in 2-5L: {low_target_achieved}/{len(low_mapes)}")
    
    print("\n" + "="*80)
    
    # Advanced model identification and ranking
    print("🏆 TOP PERFORMING MODELS IDENTIFICATION")
    print("="*80)
    
    # Filter advanced models only (exclude baselines)
    advanced_models = results_summary[
        ~results_summary['model_name'].str.contains('Baseline|Mean|Median', case=False, na=False)
    ].copy()
    
    if len(advanced_models) > 0:
        print("Advanced Models Performance:")
        print(advanced_models[available_columns].to_string(index=False))
        
        # Multi-criteria ranking system
        print(f"\n📊 MULTI-CRITERIA RANKING SYSTEM:")
        
        # Ranking criteria with weights
        ranking_df = advanced_models.copy()
        
        # Criteria 1: Overall MAPE (40% weight)
        if 'overall_mape' in ranking_df.columns:
            ranking_df['overall_rank'] = ranking_df['overall_mape'].rank()
            print(f"   🎯 Overall MAPE ranking (40% weight)")
        
        # Criteria 2: 2-5L MAPE (35% weight) - Most critical
        if 'low_mape' in ranking_df.columns:
            ranking_df['low_rank'] = ranking_df['low_mape'].rank()
            print(f"   🔴 2-5L MAPE ranking (35% weight) - CRITICAL")
        
        # Criteria 3: Training efficiency (15% weight)
        if 'training_time' in ranking_df.columns:
            ranking_df['time_rank'] = ranking_df['training_time'].rank()
            print(f"   ⏱️  Training time ranking (15% weight)")
        
        # Criteria 4: Range consistency (10% weight)
        if 'mid_mape' in ranking_df.columns and 'high_mape' in ranking_df.columns:
            ranking_df['range_consistency'] = abs(ranking_df['mid_mape'] - ranking_df['high_mape'])
            ranking_df['consistency_rank'] = ranking_df['range_consistency'].rank()
            print(f"   📏 Range consistency ranking (10% weight)")
        
        # Calculate weighted composite score
        composite_scores = []
        for idx, row in ranking_df.iterrows():
            score = 0
            weights_sum = 0
            
            if 'overall_rank' in ranking_df.columns:
                score += row['overall_rank'] * 0.40
                weights_sum += 0.40
            
            if 'low_rank' in ranking_df.columns:
                score += row['low_rank'] * 0.35
                weights_sum += 0.35
            
            if 'time_rank' in ranking_df.columns:
                score += row['time_rank'] * 0.15
                weights_sum += 0.15
            
            if 'consistency_rank' in ranking_df.columns:
                score += row['consistency_rank'] * 0.10
                weights_sum += 0.10
            
            if weights_sum > 0:
                composite_scores.append(score / weights_sum)
            else:
                composite_scores.append(float('inf'))
        
        ranking_df['composite_score'] = composite_scores
        ranking_df = ranking_df.sort_values('composite_score')
        
        print(f"\n🏆 FINAL WEIGHTED RANKING:")
        rank_display = ranking_df[['model_name', 'overall_mape', 'low_mape', 'composite_score']].copy()
        rank_display['rank'] = range(1, len(rank_display) + 1)
        rank_display = rank_display[['rank', 'model_name', 'overall_mape', 'low_mape', 'composite_score']]
        print(rank_display.to_string(index=False, float_format='%.2f'))
        
        # Select top 3 models for ensemble
        top_3_models = ranking_df.head(3)
        
        print(f"\n🎖️  TOP 3 MODELS SELECTED FOR ENSEMBLE:")
        for i, (idx, model) in enumerate(top_3_models.iterrows(), 1):
            print(f"   {i}. {model['model_name']}")
            print(f"      📊 Overall MAPE: {model['overall_mape']:.2f}%")
            if 'low_mape' in model:
                print(f"      🔴 2-5L MAPE: {model['low_mape']:.2f}%")
            if 'training_time' in model:
                print(f"      ⏱️  Training Time: {model['training_time']:.1f}s")
            print(f"      🏆 Composite Score: {model['composite_score']:.2f}")
            print()
        
        # Best individual model analysis
        best_model = ranking_df.iloc[0]
        print(f"🥇 BEST INDIVIDUAL MODEL: {best_model['model_name']}")
        print(f"   🎯 Achieves overall MAPE: {best_model['overall_mape']:.2f}%")
        
        if best_model['overall_mape'] < 18:
            print(f"   ✅ SUCCESS: Achieved target <18% MAPE!")
        else:
            gap = best_model['overall_mape'] - 18
            print(f"   ⚠️  Gap to target: +{gap:.2f}% (ensemble may close this gap)")
        
        if 'low_mape' in best_model:
            if best_model['low_mape'] < 25:
                print(f"   ✅ SUCCESS: 2-5L range improved to {best_model['low_mape']:.2f}%")
            else:
                gap_low = best_model['low_mape'] - 25
                print(f"   ⚠️  2-5L gap to target: +{gap_low:.2f}%")
    
    else:
        print("❌ No advanced models found for ranking")
    
    # Visualization of model comparison
    if len(results_summary) >= 3:
        plt.figure(figsize=(16, 10))
        
        # Overall MAPE comparison
        plt.subplot(2, 3, 1)
        models = results_summary['model_name']
        overall_mapes = results_summary['overall_mape']
        
        colors = ['red' if mape >= 18 else 'green' for mape in overall_mapes]
        bars = plt.bar(range(len(models)), overall_mapes, color=colors, alpha=0.7)
        plt.axhline(y=18, color='red', linestyle='--', alpha=0.8, label='Target: 18%')
        plt.xlabel('Models')
        plt.ylabel('Overall MAPE (%)')
        plt.title('Overall MAPE Comparison')
        plt.xticks(range(len(models)), [m[:10] + '...' if len(m) > 10 else m for m in models], rotation=45)
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 2-5L MAPE comparison
        if 'low_mape' in results_summary.columns:
            plt.subplot(2, 3, 2)
            low_mapes = results_summary['low_mape'].fillna(0)
            colors_low = ['red' if mape >= 25 else 'green' for mape in low_mapes]
            plt.bar(range(len(models)), low_mapes, color=colors_low, alpha=0.7)
            plt.axhline(y=25, color='red', linestyle='--', alpha=0.8, label='Target: 25%')
            plt.xlabel('Models')
            plt.ylabel('2-5L MAPE (%)')
            plt.title('2-5L Range MAPE Comparison')
            plt.xticks(range(len(models)), [m[:10] + '...' if len(m) > 10 else m for m in models], rotation=45)
            plt.legend()
            plt.grid(True, alpha=0.3)
        
        # Training time comparison
        if 'training_time' in results_summary.columns:
            plt.subplot(2, 3, 3)
            times = results_summary['training_time'].fillna(0)
            plt.bar(range(len(models)), times, alpha=0.7, color='blue')
            plt.xlabel('Models')
            plt.ylabel('Training Time (seconds)')
            plt.title('Training Time Comparison')
            plt.xticks(range(len(models)), [m[:10] + '...' if len(m) > 10 else m for m in models], rotation=45)
            plt.grid(True, alpha=0.3)
        
        # Range performance radar chart for top 3
        if len(advanced_models) >= 3 and all(col in advanced_models.columns for col in ['low_mape', 'mid_mape', 'high_mape']):
            plt.subplot(2, 3, 4)
            top_3 = advanced_models.head(3)
            
            ranges = ['2-5L', '5-10L', '10L+']
            angles = np.linspace(0, 2 * np.pi, len(ranges), endpoint=False)
            angles = np.concatenate((angles, [angles[0]]))
            
            for i, (idx, model) in enumerate(top_3.iterrows()):
                values = [model['low_mape'], model['mid_mape'], model['high_mape']]
                values = np.concatenate((values, [values[0]]))
                plt.polar(angles, values, 'o-', linewidth=2, label=model['model_name'][:15])
            
            plt.thetagrids(angles[:-1] * 180/np.pi, ranges)
            plt.title('Top 3 Models: Range Performance')
            plt.legend()
        
        # Performance improvement timeline
        plt.subplot(2, 3, 5)
        if 'timestamp' in results_summary.columns:
            sorted_results = results_summary.sort_values('timestamp')
            cumulative_best = sorted_results['overall_mape'].cummin()
            plt.plot(range(len(cumulative_best)), cumulative_best, 'b-o', alpha=0.7)
            plt.axhline(y=18, color='red', linestyle='--', alpha=0.8, label='Target: 18%')
            plt.xlabel('Model Sequence')
            plt.ylabel('Best MAPE So Far (%)')
            plt.title('Performance Improvement Timeline')
            plt.legend()
            plt.grid(True, alpha=0.3)
        
        # Model efficiency scatter plot
        if 'training_time' in results_summary.columns:
            plt.subplot(2, 3, 6)
            plt.scatter(results_summary['training_time'], results_summary['overall_mape'], 
                       alpha=0.7, s=100, c=range(len(results_summary)), cmap='viridis')
            
            for i, model in enumerate(results_summary['model_name']):
                plt.annotate(model[:8], 
                           (results_summary.iloc[i]['training_time'], results_summary.iloc[i]['overall_mape']),
                           xytext=(5, 5), textcoords='offset points', fontsize=8)
            
            plt.xlabel('Training Time (seconds)')
            plt.ylabel('Overall MAPE (%)')
            plt.title('Model Efficiency: MAPE vs Training Time')
            plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    print("\n" + "="*80)
    print("✅ Model comparison and selection complete!")
    print("🎯 Ready for ensemble methods with top performing models")
    print("="*80)
    
else:
    print("❌ No model results available for comparison")

print("=" * 60)

In [ ]:
# =============================================================================
# CHUNK 11: BEST MODEL DEEP ANALYSIS
# =============================================================================

print("🔍 Deep Analysis of Best Performing Model")
print("🎯 Objective: Understand model behavior, strengths, and limitations")

# Get the best model
best_model_info = model_tracker.get_best_model()

if best_model_info is not None:
    best_model_name = best_model_info['model_name']
    best_overall_mape = best_model_info['overall_mape']
    
    print(f"\n🏆 ANALYZING BEST MODEL: {best_model_name}")
    print(f"🎯 Overall MAPE: {best_overall_mape:.2f}%")
    print("="*70)
    
    # Load the best model for detailed analysis
    best_model = None
    best_model_path = None
    feature_importance_df = None
    
    # Determine model path and load
    if 'LightGBM' in best_model_name:
        best_model_path = MODELS_DIR / "lightgbm_optimized.pkl"
        if best_model_path.exists():
            with open(best_model_path, 'rb') as f:
                model_data = pickle.load(f)
                best_model = model_data['model']
                feature_importance_df = model_data.get('feature_importance', None)
            
            # Generate predictions for analysis
            best_pred_log = best_model.predict(X_val, num_iteration=best_model.best_iteration)
            
    elif 'XGBoost' in best_model_name:
        best_model_path = MODELS_DIR / "xgboost_optimized.pkl"
        if best_model_path.exists():
            with open(best_model_path, 'rb') as f:
                model_data = pickle.load(f)
                best_model = model_data['model']
                feature_importance_df = model_data.get('feature_importance', None)
            
            best_pred_log = best_model.predict(X_val)
            
    elif 'CatBoost' in best_model_name:
        best_model_path = MODELS_DIR / "catboost_optimized.cbm"
        metadata_path = MODELS_DIR / "catboost_optimized_metadata.pkl"
        
        if best_model_path.exists():
            best_model = cb.CatBoostRegressor()
            best_model.load_model(str(best_model_path))
            
            if metadata_path.exists():
                with open(metadata_path, 'rb') as f:
                    metadata = pickle.load(f)
                    feature_importance_df = metadata.get('feature_importance', None)
            
            best_pred_log = best_model.predict(X_val)
            
    elif 'Extra Trees' in best_model_name:
        best_model_path = MODELS_DIR / "extratrees_optimized.pkl"
        if best_model_path.exists():
            with open(best_model_path, 'rb') as f:
                model_data = pickle.load(f)
                best_model = model_data['model']
                feature_importance_df = model_data.get('feature_importance', None)
            
            best_pred_log = best_model.predict(X_val)
    
    if best_model is not None:
        # Convert predictions to original scale
        best_pred_orig = np.expm1(best_pred_log)
        y_val_orig = np.expm1(y_val_log)
        
        # 1. DETAILED PERFORMANCE BREAKDOWN
        print("📊 DETAILED PERFORMANCE BREAKDOWN")
        print("-" * 50)
        
        # Overall metrics
        overall_mae = mean_absolute_error(y_val_orig, best_pred_orig)
        overall_rmse = np.sqrt(mean_squared_error(y_val_orig, best_pred_orig))
        overall_r2 = r2_score(y_val_orig, best_pred_orig)
        
        print(f"Overall Performance:")
        print(f"   🎯 MAPE: {best_overall_mape:.2f}%")
        print(f"   📏 MAE:  ₹{overall_mae:,.0f}")
        print(f"   📐 RMSE: ₹{overall_rmse:,.0f}")
        print(f"   📈 R²:   {overall_r2:.4f}")
        
        # Range-specific detailed analysis
        print(f"\nIncome Range Analysis:")
        range_labels = {'low': '2-5L', 'mid': '5-10L', 'high': '10L+'}
        range_details = {}
        
        for range_key, mask in income_ranges_val.items():
            if mask.sum() > 0:
                range_mape = calculate_mape(y_val_orig[mask], best_pred_orig[mask])
                range_mae = mean_absolute_error(y_val_orig[mask], best_pred_orig[mask])
                range_rmse = np.sqrt(mean_squared_error(y_val_orig[mask], best_pred_orig[mask]))
                range_r2 = r2_score(y_val_orig[mask], best_pred_orig[mask])
                range_count = mask.sum()
                
                range_details[range_key] = {
                    'mape': range_mape, 'mae': range_mae, 'rmse': range_rmse,
                    'r2': range_r2, 'count': range_count
                }
                
                status = "✅" if range_mape < 20 else "⚠️" if range_mape < 25 else "❌"
                print(f"   {status} {range_labels[range_key]:>4}: {range_mape:>6.2f}% MAPE | ₹{range_mae:>8,.0f} MAE | R²={range_r2:>6.3f} | {range_count:>6,} samples")
        
        # 2. FEATURE IMPORTANCE ANALYSIS
        print(f"\n📊 FEATURE IMPORTANCE ANALYSIS")
        print("-" * 50)
        
        if feature_importance_df is not None:
            top_20_features = feature_importance_df.head(20)
            print("Top 20 Most Important Features:")
            
            for i, (_, row) in enumerate(top_20_features.iterrows(), 1):
                feature_name = row['feature']
                importance = row['importance']
                
                # Categorize feature type
                if any(keyword in feature_name.lower() for keyword in ['crop', 'production', 'yield']):
                    category = "🌾 Agricultural"
                elif any(keyword in feature_name.lower() for keyword in ['weather', 'temp', 'rain']):
                    category = "🌤️  Weather"
                elif any(keyword in feature_name.lower() for keyword in ['income', 'financial', 'profit']):
                    category = "💰 Financial"
                elif any(keyword in feature_name.lower() for keyword in ['geo', 'location', 'distance']):
                    category = "📍 Geographic"
                elif any(keyword in feature_name.lower() for keyword in ['age', 'education', 'family']):
                    category = "👥 Demographic"
                else:
                    category = "🔧 Engineered"
                
                print(f"   {i:>2}. {category} | {feature_name[:40]:.<40} {importance:>8.3f}")
            
            # Feature category analysis
            print(f"\nFeature Category Distribution (Top 20):")
            categories = {}
            for _, row in top_20_features.iterrows():
                feature_name = row['feature']
                if any(keyword in feature_name.lower() for keyword in ['crop', 'production', 'yield']):
                    cat = "Agricultural"
                elif any(keyword in feature_name.lower() for keyword in ['weather', 'temp', 'rain']):
                    cat = "Weather"
                elif any(keyword in feature_name.lower() for keyword in ['income', 'financial', 'profit']):
                    cat = "Financial"
                elif any(keyword in feature_name.lower() for keyword in ['geo', 'location', 'distance']):
                    cat = "Geographic"
                elif any(keyword in feature_name.lower() for keyword in ['age', 'education', 'family']):
                    cat = "Demographic"
                else:
                    cat = "Engineered"
                
                categories[cat] = categories.get(cat, 0) + 1
            
            for cat, count in sorted(categories.items(), key=lambda x: x[1], reverse=True):
                print(f"   🔹 {cat}: {count} features")
        
        # 3. ERROR ANALYSIS BY SEGMENTS
        print(f"\n📊 ERROR ANALYSIS BY SEGMENTS")
        print("-" * 50)
        
        # Calculate errors
        errors = np.abs(y_val_orig - best_pred_orig)
        percentage_errors = errors / y_val_orig * 100
        
        # Error quantile analysis
        print("Error Distribution Analysis:")
        percentiles = [10, 25, 50, 75, 90, 95, 99]
        print("   📊 Percentage Error Percentiles:")
        for p in percentiles:
            value = np.percentile(percentage_errors, p)
            print(f"      P{p:>2}: {value:>6.2f}%")
        
        # High error cases analysis
        high_error_threshold = np.percentile(percentage_errors, 95)
        high_error_mask = percentage_errors > high_error_threshold
        
        print(f"\n🔍 High Error Cases Analysis (>{high_error_threshold:.1f}% error):")
        print(f"   📊 Total high error cases: {high_error_mask.sum():,} ({high_error_mask.sum()/len(y_val_orig)*100:.1f}%)")
        
        # Analyze high error cases by income range
        for range_key, range_mask in income_ranges_val.items():
            if range_mask.sum() > 0:
                high_error_in_range = (high_error_mask & range_mask).sum()
                range_total = range_mask.sum()
                pct_high_error = high_error_in_range / range_total * 100
                print(f"   🔹 {range_labels[range_key]:>4}: {high_error_in_range:>4,}/{range_total:>5,} ({pct_high_error:>5.1f}%) high error cases")
        
        # 4. PREDICTION PATTERN ANALYSIS
        print(f"\n📊 PREDICTION PATTERN ANALYSIS")
        print("-" * 50)
        
        # Bias analysis
        bias = np.mean(best_pred_orig - y_val_orig)
        abs_bias = np.mean(np.abs(best_pred_orig - y_val_orig))
        
        print(f"Prediction Bias Analysis:")
        print(f"   🎯 Mean Bias: ₹{bias:,.0f} ({'Over' if bias > 0 else 'Under'}prediction)")
        print(f"   📏 Mean Absolute Bias: ₹{abs_bias:,.0f}")
        
        # Range-specific bias
        print(f"\nBias by Income Range:")
        for range_key, mask in income_ranges_val.items():
            if mask.sum() > 0:
                range_bias = np.mean(best_pred_orig[mask] - y_val_orig[mask])
                direction = "Over" if range_bias > 0 else "Under"
                print(f"   🔹 {range_labels[range_key]:>4}: ₹{range_bias:>8,.0f} ({direction}prediction)")
        
        # 5. MODEL STRENGTHS AND LIMITATIONS
        print(f"\n📊 MODEL STRENGTHS AND LIMITATIONS")
        print("-" * 50)
        
        print("🟢 STRENGTHS:")
        
        # Identify strengths based on performance
        if best_overall_mape < 18:
            print("   ✅ Achieves target overall MAPE < 18%")
        
        best_range_performance = min([range_details[k]['mape'] for k in range_details.keys()])
        if best_range_performance < 20:
            print("   ✅ Strong performance in at least one income range")
        
        if overall_r2 > 0.8:
            print("   ✅ High R² indicates good variance explanation")
        
        if 'low' in range_details and range_details['low']['mape'] < 30:
            print("   ✅ Improved 2-5L range performance from baseline")
        
        # Analyze prediction consistency
        cv_std = np.std([range_details[k]['mape'] for k in range_details.keys()])
        if cv_std < 5:
            print("   ✅ Consistent performance across income ranges")
        
        print("\n🔴 LIMITATIONS:")
        
        # Identify limitations
        if best_overall_mape >= 18:
            gap = best_overall_mape - 18
            print(f"   ⚠️  Overall MAPE {gap:.2f}% above target")
        
        if 'low' in range_details and range_details['low']['mape'] > 25:
            gap_low = range_details['low']['mape'] - 25
            print(f"   ⚠️  2-5L range MAPE {gap_low:.2f}% above target")
        
        if bias > 50000:  # Significant bias
            print(f"   ⚠️  Systematic overprediction bias of ₹{bias:,.0f}")
        elif bias < -50000:
            print(f"   ⚠️  Systematic underprediction bias of ₹{abs(bias):,.0f}")
        
        high_error_rate = high_error_mask.sum() / len(y_val_orig) * 100
        if high_error_rate > 10:
            print(f"   ⚠️  High error rate: {high_error_rate:.1f}% of cases have >{high_error_threshold:.1f}% error")
        
        # 6. VISUALIZATION
        plt.figure(figsize=(20, 12))
        
        # Predictions vs Actual by range
        plt.subplot(2, 4, 1)
        colors = {'low': 'red', 'mid': 'blue', 'high': 'green'}
        for range_key, mask in income_ranges_val.items():
            if mask.sum() > 0:
                y_true_millions = y_val_orig[mask] / 1000000
                y_pred_millions = best_pred_orig[mask] / 1000000
                plt.scatter(y_true_millions, y_pred_millions, 
                           alpha=0.6, c=colors[range_key], label=range_labels[range_key], s=15)
        
        max_val = max(y_val_orig.max(), best_pred_orig.max()) / 1000000
        min_val = min(y_val_orig.min(), best_pred_orig.min()) / 1000000
        plt.plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.8, linewidth=2)
        plt.xlabel('Actual Income (₹ Millions)')
        plt.ylabel('Predicted Income (₹ Millions)')
        plt.title(f'{best_model_name}: Predictions vs Actual')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Error distribution
        plt.subplot(2, 4, 2)
        plt.hist(percentage_errors, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
        plt.axvline(best_overall_mape, color='red', linestyle='--', label=f'Mean: {best_overall_mape:.1f}%')
        plt.axvline(18, color='green', linestyle='--', label='Target: 18%')
        plt.xlabel('Absolute Percentage Error (%)')
        plt.ylabel('Frequency')
        plt.title('Error Distribution')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Residuals plot
        plt.subplot(2, 4, 3)
        residuals = best_pred_orig - y_val_orig
        plt.scatter(best_pred_orig/1000000, residuals/1000000, alpha=0.6, s=15)
        plt.axhline(y=0, color='red', linestyle='--', alpha=0.8)
        plt.xlabel('Predicted Income (₹ Millions)')
        plt.ylabel('Residuals (₹ Millions)')
        plt.title('Residuals Plot')
        plt.grid(True, alpha=0.3)
        
        # Feature importance
        if feature_importance_df is not None:
            plt.subplot(2, 4, 4)
            top_10 = feature_importance_df.head(10)
            plt.barh(range(len(top_10)), top_10['importance'])
            plt.yticks(range(len(top_10)), [f[:20] + '...' if len(f) > 20 else f for f in top_10['feature']])
            plt.xlabel('Importance')
            plt.title('Top 10 Feature Importance')
            plt.grid(True, alpha=0.3)
        
        # Error by income range
        plt.subplot(2, 4, 5)
        range_mapes = [range_details[k]['mape'] for k in ['low', 'mid', 'high']]
        range_names = ['2-5L', '5-10L', '10L+']
        colors_bar = ['red' if x > 20 else 'green' for x in range_mapes]
        bars = plt.bar(range_names, range_mapes, color=colors_bar, alpha=0.7)
        plt.axhline(y=18, color='green', linestyle='--', alpha=0.8, label='Overall Target')
        plt.axhline(y=25, color='orange', linestyle='--', alpha=0.8, label='2-5L Target')
        plt.ylabel('MAPE (%)')
        plt.title('MAPE by Income Range')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Prediction accuracy by actual income
        plt.subplot(2, 4, 6)
        income_bins_viz = np.linspace(y_val_orig.min(), y_val_orig.max(), 20)
        bin_centers = (income_bins_viz[:-1] + income_bins_viz[1:]) / 2
        bin_mapes = []
        
        for i in range(len(income_bins_viz)-1):
            mask = (y_val_orig >= income_bins_viz[i]) & (y_val_orig < income_bins_viz[i+1])
            if mask.sum() > 10:  # Enough samples
                bin_mape = calculate_mape(y_val_orig[mask], best_pred_orig[mask])
                bin_mapes.append(bin_mape)
            else:
                bin_mapes.append(np.nan)
        
        valid_bins = ~np.isnan(bin_mapes)
        plt.plot(bin_centers[valid_bins]/1000000, np.array(bin_mapes)[valid_bins], 'b-o', alpha=0.7)
        plt.axhline(y=18, color='red', linestyle='--', alpha=0.8, label='Target: 18%')
        plt.xlabel('Actual Income (₹ Millions)')
        plt.ylabel('MAPE (%)')
        plt.title('Accuracy vs Income Level')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Training time vs performance comparison
        plt.subplot(2, 4, 7)
        if len(model_tracker.results) > 1:
            all_times = [r.get('training_time', 0) for r in model_tracker.results]
            all_mapes = [r['overall_mape'] for r in model_tracker.results]
            all_names = [r['model_name'] for r in model_tracker.results]
            
            plt.scatter(all_times, all_mapes, alpha=0.7, s=100)
            
            # Highlight best model
            best_idx = all_names.index(best_model_name)
            plt.scatter(all_times[best_idx], all_mapes[best_idx], 
                       color='red', s=200, marker='*', label='Best Model')
            
            plt.xlabel('Training Time (seconds)')
            plt.ylabel('Overall MAPE (%)')
            plt.title('Efficiency: Performance vs Time')
            plt.legend()
            plt.grid(True, alpha=0.3)
        
        # Model confidence intervals (if applicable)
        plt.subplot(2, 4, 8)
        sorted_indices = np.argsort(y_val_orig)
        sorted_actual = y_val_orig[sorted_indices]
        sorted_pred = best_pred_orig[sorted_indices]
        
        # Calculate moving average for smoothing
        window = len(sorted_actual) // 50
        if window > 1:
            smooth_actual = np.convolve(sorted_actual, np.ones(window)/window, mode='valid')
            smooth_pred = np.convolve(sorted_pred, np.ones(window)/window, mode='valid')
            
            plt.plot(smooth_actual/1000000, smooth_pred/1000000, 'b-', alpha=0.8, label='Predicted')
            plt.plot(smooth_actual/1000000, smooth_actual/1000000, 'r--', alpha=0.8, label='Perfect')
            plt.xlabel('Actual Income (₹ Millions)')
            plt.ylabel('Predicted Income (₹ Millions)')
            plt.title('Prediction Smoothness')
            plt.legend()
            plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # 7. BUSINESS INSIGHTS
        print(f"\n💼 BUSINESS INSIGHTS")
        print("-" * 50)
        
        if feature_importance_df is not None:
            top_5_features = feature_importance_df.head(5)['feature'].tolist()
            
            print("Key Predictive Factors:")
            for i, feature in enumerate(top_5_features, 1):
                if 'crop' in feature.lower() or 'production' in feature.lower():
                    insight = "Agricultural productivity is a key income driver"
                elif 'weather' in feature.lower() or 'rain' in feature.lower():
                    insight = "Weather patterns significantly impact farmer income"
                elif 'financial' in feature.lower() or 'income' in feature.lower():
                    insight = "Financial diversification affects income stability"
                elif 'geo' in feature.lower() or 'location' in feature.lower():
                    insight = "Geographic location influences income potential"
                else:
                    insight = "This factor contributes to income prediction"
                
                print(f"   {i}. {feature[:30]:<30} → {insight}")
        
        print(f"\nRecommendations for Income Improvement:")
        
        if 'low' in range_details and range_details['low']['mape'] > 25:
            print("   🎯 Focus interventions on 2-5L income farmers (highest prediction error)")
        
        print("   🌾 Enhance agricultural productivity programs")
        print("   🌤️  Improve weather resilience and forecasting")
        print("   💰 Promote income diversification strategies")
        print("   📍 Target location-specific interventions")
        
        print("\n" + "="*70)
        print(f"✅ Deep analysis of {best_model_name} complete!")
        print("🎯 Model ready for ensemble combination and final predictions")
        print("="*70)
        
    else:
        print(f"❌ Could not load best model from {best_model_path}")
        print("   Please ensure model files are properly saved")

else:
    print("❌ No best model found for analysis")

print("=" * 60)

In [ ]:
# =============================================================================
# CHUNK 12: FINAL VALIDATION & TEST PREDICTIONS
# =============================================================================

print("🎯 Final Validation & Test Predictions Generation")
print("🏆 Objective: Validate best models and generate submission predictions")

# Get top 3 models for final validation
results_summary = model_tracker.get_summary()
if results_summary is not None and len(results_summary) > 0:
    
    # Filter advanced models and get top 3
    advanced_models = results_summary[
        ~results_summary['model_name'].str.contains('Baseline|Mean|Median', case=False, na=False)
    ].copy()
    
    if len(advanced_models) >= 3:
        top_3_models = advanced_models.head(3)
        model_names = top_3_models['model_name'].tolist()
        
        print(f"\n🏆 TOP 3 MODELS FOR FINAL VALIDATION:")
        for i, name in enumerate(model_names, 1):
            mape = top_3_models.iloc[i-1]['overall_mape']
            print(f"   {i}. {name}: {mape:.2f}% MAPE")
        
        print("\n" + "="*70)
        
        # Load models and generate predictions
        final_models = {}
        final_predictions_val = {}
        final_predictions_test = {}
        
        print("📊 LOADING MODELS AND GENERATING PREDICTIONS")
        print("-" * 50)
        
        for model_name in model_names:
            print(f"\n🔄 Processing {model_name}...")
            
            try:
                if 'LightGBM' in model_name:
                    model_path = MODELS_DIR / "lightgbm_optimized.pkl"
                    with open(model_path, 'rb') as f:
                        model_data = pickle.load(f)
                        model = model_data['model']
                    
                    pred_val_log = model.predict(X_val, num_iteration=model.best_iteration)
                    pred_test_log = model.predict(X_test, num_iteration=model.best_iteration)
                    
                elif 'XGBoost' in model_name:
                    model_path = MODELS_DIR / "xgboost_optimized.pkl"
                    with open(model_path, 'rb') as f:
                        model_data = pickle.load(f)
                        model = model_data['model']
                    
                    pred_val_log = model.predict(X_val)
                    pred_test_log = model.predict(X_test)
                    
                elif 'CatBoost' in model_name:
                    model_path = MODELS_DIR / "catboost_optimized.cbm"
                    model = cb.CatBoostRegressor()
                    model.load_model(str(model_path))
                    
                    pred_val_log = model.predict(X_val)
                    pred_test_log = model.predict(X_test)
                    
                elif 'Extra Trees' in model_name:
                    model_path = MODELS_DIR / "extratrees_optimized.pkl"
                    with open(model_path, 'rb') as f:
                        model_data = pickle.load(f)
                        model = model_data['model']
                    
                    pred_val_log = model.predict(X_val)
                    pred_test_log = model.predict(X_test)
                
                # Store models and predictions
                final_models[model_name] = model
                final_predictions_val[model_name] = np.expm1(pred_val_log)  # Convert to original scale
                final_predictions_test[model_name] = np.expm1(pred_test_log)  # Convert to original scale
                
                # Validate predictions
                val_mape = calculate_mape(np.expm1(y_val_log), final_predictions_val[model_name])
                print(f"   ✅ Validation MAPE: {val_mape:.2f}%")
                print(f"   📊 Test predictions range: ₹{final_predictions_test[model_name].min():,.0f} - ₹{final_predictions_test[model_name].max():,.0f}")
                
            except Exception as e:
                print(f"   ❌ Error loading {model_name}: {str(e)}")
                continue
        
        print(f"\n✅ Successfully loaded {len(final_models)} models")
        
        # CROSS-VALIDATION ON BEST MODEL
        print("\n📊 CROSS-VALIDATION ON BEST MODEL")
        print("-" * 50)
        
        best_model_name = model_names[0]  # Top model
        if best_model_name in final_models:
            print(f"🔄 Performing 5-fold CV on {best_model_name}...")
            
            cv_scores = []
            cv_range_scores = {'low': [], 'mid': [], 'high': []}
            
            for fold, (train_idx, val_idx) in enumerate(cv_splitter.split(X_train, income_bins), 1):
                X_cv_train, X_cv_val = X_train[train_idx], X_train[val_idx]
                y_cv_train, y_cv_val = y_train_log[train_idx], y_train_log[val_idx]
                weights_cv_train = sample_weights[train_idx]
                
                # Train model on fold
                if 'LightGBM' in best_model_name:
                    train_data = lgb.Dataset(X_cv_train, label=y_cv_train, weight=weights_cv_train)
                    cv_model = lgb.train(
                        final_models[best_model_name].params,
                        train_data,
                        num_boost_round=1000,
                        valid_sets=[train_data],
                        callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)]
                    )
                    cv_pred_log = cv_model.predict(X_cv_val, num_iteration=cv_model.best_iteration)
                    
                elif 'XGBoost' in best_model_name:
                    cv_model = xgb.XGBRegressor(**final_models[best_model_name].get_params())
                    cv_model.fit(X_cv_train, y_cv_train, sample_weight=weights_cv_train,
                               eval_set=[(X_cv_train, y_cv_train)], early_stopping_rounds=100, verbose=False)
                    cv_pred_log = cv_model.predict(X_cv_val)
                    
                elif 'CatBoost' in best_model_name:
                    train_pool = cb.Pool(X_cv_train, y_cv_train, weight=weights_cv_train)
                    cv_model = cb.CatBoostRegressor(**final_models[best_model_name].get_all_params())
                    cv_model.fit(train_pool, verbose=False)
                    cv_pred_log = cv_model.predict(X_cv_val)
                    
                elif 'Extra Trees' in best_model_name:
                    cv_model = ExtraTreesRegressor(**final_models[best_model_name].get_params())
                    cv_model.fit(X_cv_train, y_cv_train, sample_weight=weights_cv_train)
                    cv_pred_log = cv_model.predict(X_cv_val)
                
                # Evaluate fold
                cv_true_orig = np.expm1(y_cv_val)
                cv_pred_orig = np.expm1(cv_pred_log)
                fold_mape = calculate_mape(cv_true_orig, cv_pred_orig)
                cv_scores.append(fold_mape)
                
                # Range-specific evaluation
                for range_key, range_mask_full in income_ranges.items():
                    # Map range mask to current fold
                    range_mask_fold = range_mask_full[val_idx]
                    if range_mask_fold.sum() > 0:
                        range_mape = calculate_mape(cv_true_orig[range_mask_fold], cv_pred_orig[range_mask_fold])
                        cv_range_scores[range_key].append(range_mape)
                
                print(f"   Fold {fold}: {fold_mape:.2f}% MAPE")
            
            # CV Results Summary
            cv_mean = np.mean(cv_scores)
            cv_std = np.std(cv_scores)
            
            print(f"\n📊 Cross-Validation Results:")
            print(f"   🎯 Mean MAPE: {cv_mean:.2f} ± {cv_std:.2f}%")
            print(f"   📊 Individual folds: {[f'{score:.2f}%' for score in cv_scores]}")
            
            # Range-specific CV results
            print(f"\n🎯 Range-Specific CV Results:")
            range_labels = {'low': '2-5L', 'mid': '5-10L', 'high': '10L+'}
            for range_key, scores in cv_range_scores.items():
                if scores:
                    range_mean = np.mean(scores)
                    range_std = np.std(scores)
                    print(f"   🔹 {range_labels[range_key]:>4}: {range_mean:.2f} ± {range_std:.2f}%")
            
            # Statistical significance test
            from scipy import stats
            if len(cv_scores) >= 3:
                # Test if significantly different from 18%
                t_stat, p_value = stats.ttest_1samp(cv_scores, 18)
                print(f"\n📈 Statistical Analysis:")
                print(f"   🧮 T-test vs 18% target: t={t_stat:.3f}, p={p_value:.3f}")
                if p_value < 0.05:
                    if cv_mean < 18:
                        print(f"   ✅ Significantly better than 18% target (p<0.05)")
                    else:
                        print(f"   ⚠️  Significantly worse than 18% target (p<0.05)")
                else:
                    print(f"   📊 No significant difference from 18% target (p≥0.05)")
        
        # ENSEMBLE PREDICTIONS
        print("\n🤖 SIMPLE ENSEMBLE PREDICTIONS")
        print("-" * 50)
        
        if len(final_predictions_val) >= 2:
            # Simple average ensemble
            ensemble_val = np.mean(list(final_predictions_val.values()), axis=0)
            ensemble_test = np.mean(list(final_predictions_test.values()), axis=0)
            
            # Evaluate ensemble
            ensemble_mape = calculate_mape(np.expm1(y_val_log), ensemble_val)
            
            print(f"📊 Simple Average Ensemble:")
            print(f"   🎯 Validation MAPE: {ensemble_mape:.2f}%")
            
            # Range-specific ensemble evaluation
            ensemble_range_metrics = {}
            for range_key, mask in income_ranges_val.items():
                if mask.sum() > 0:
                    range_mape = calculate_mape(np.expm1(y_val_log)[mask], ensemble_val[mask])
                    ensemble_range_metrics[range_key] = range_mape
                    print(f"   🔹 {range_labels[range_key]:>4}: {range_mape:.2f}% MAPE")
            
            # Store ensemble results
            final_predictions_val['Simple Ensemble'] = ensemble_val
            final_predictions_test['Simple Ensemble'] = ensemble_test
            
            # Track ensemble performance
            model_tracker.add_result("Simple Ensemble", ensemble_mape, ensemble_range_metrics)
        
        # GENERATE FINAL SUBMISSION FILES
        print("\n💾 GENERATING SUBMISSION FILES")
        print("-" * 50)
        
        submissions_dir = RESULTS_DIR / "submissions"
        submissions_dir.mkdir(exist_ok=True)
        
        for model_name, test_predictions in final_predictions_test.items():
            # Create submission dataframe
            submission_df = pd.DataFrame({
                'id': range(len(test_predictions)),
                'predicted_income': test_predictions.round(2)
            })
            
            # Save submission file
            filename = f"submission_{model_name.lower().replace(' ', '_')}.csv"
            file_path = submissions_dir / filename
            submission_df.to_csv(file_path, index=False)
            
            print(f"   📄 {model_name}: {file_path}")
            print(f"      📊 Predictions range: ₹{test_predictions.min():,.0f} - ₹{test_predictions.max():,.0f}")
            print(f"      📈 Mean prediction: ₹{test_predictions.mean():,.0f}")
        
        # FINAL RECOMMENDATIONS
        print("\n🏆 FINAL MODEL RECOMMENDATIONS")
        print("-" * 50)
        
        # Get final best model
        final_best = model_tracker.get_best_model()
        if final_best:
            print(f"🥇 RECOMMENDED MODEL: {final_best['model_name']}")
            print(f"   🎯 Overall MAPE: {final_best['overall_mape']:.2f}%")
            
            if final_best['overall_mape'] < 18:
                print(f"   ✅ MISSION ACCOMPLISHED: Target <18% MAPE achieved!")
            else:
                gap = final_best['overall_mape'] - 18
                print(f"   📊 Gap to target: +{gap:.2f}% MAPE")
            
            # 2-5L range assessment
            if 'low_mape' in final_best and final_best['low_mape'] is not None:
                print(f"   🔴 2-5L range MAPE: {final_best['low_mape']:.2f}%")
                if final_best['low_mape'] < 25:
                    print(f"   ✅ 2-5L range target achieved!")
                else:
                    gap_low = final_best['low_mape'] - 25
                    print(f"   📊 2-5L gap to target: +{gap_low:.2f}%")
        
        print(f"\n📋 SUBMISSION STRATEGY:")
        print(f"   🎯 Primary: Best individual model ({final_best['model_name'] if final_best else 'Unknown'})")
        if 'Simple Ensemble' in final_predictions_test:
            ensemble_performance = [r for r in model_tracker.results if r['model_name'] == 'Simple Ensemble']
            if ensemble_performance:
                ensemble_mape = ensemble_performance[0]['overall_mape']
                print(f"   🤖 Alternative: Simple ensemble ({ensemble_mape:.2f}% MAPE)")
        print(f"   📈 Backup: Top 3 individual models available")
        
        # FINAL STATISTICS
        print(f"\n📊 FINAL EXPERIMENT STATISTICS")
        print("-" * 50)
        print(f"   🔢 Total models trained: {len(model_tracker.results)}")
        print(f"   ⏱️  Total training time: {sum(r.get('training_time', 0) for r in model_tracker.results):.1f} seconds")
        print(f"   🎯 Models achieving <18% MAPE: {sum(1 for r in model_tracker.results if r['overall_mape'] < 18)}")
        print(f"   🔴 Models achieving <25% 2-5L MAPE: {sum(1 for r in model_tracker.results if r.get('low_mape', float('inf')) < 25)}")
        
        # Success assessment
        success_criteria = {
            'target_mape': final_best['overall_mape'] < 18 if final_best else False,
            'range_improvement': final_best.get('low_mape', 50) < 35 if final_best else False,  # Significant improvement from 41%
            'models_trained': len(model_tracker.results) >= 8,
            'ensemble_created': 'Simple Ensemble' in final_predictions_test
        }
        
        success_count = sum(success_criteria.values())
        total_criteria = len(success_criteria)
        
        print(f"\n🏆 SUCCESS ASSESSMENT: {success_count}/{total_criteria} criteria met")
        for criterion, achieved in success_criteria.items():
            status = "✅" if achieved else "❌"
            print(f"   {status} {criterion.replace('_', ' ').title()}")
        
        if success_count >= 3:
            print(f"\n🎉 EXPERIMENT SUCCESSFUL!")
            print(f"   🚀 Ready for production deployment")
        else:
            print(f"\n📊 EXPERIMENT PARTIALLY SUCCESSFUL")
            print(f"   🔧 Consider advanced ensemble methods in next notebook")
        
    else:
        print("❌ Insufficient advanced models for final validation")

else:
    print("❌ No model results available for final validation")

print("\n" + "="*70)
print("✅ MODELING EXPERIMENTS COMPLETE!")
print("🎯 All predictions generated and saved")
print("📁 Check results/submissions/ for submission files")
print("🚀 Proceed to Notebook 5 for advanced ensemble methods")
print("="*70)